In [81]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.pipeline import Pipeline
import joblib

In [82]:
# from google.colab import files
# uploaded = files.upload()

In [83]:
dataset_pca = pd.read_csv("multi_modeling_pca.csv")

In [84]:
dataset_MI = pd.read_csv("multi_modeling_mutl_info.csv")

In [85]:
dataset_rfe = pd.read_csv("multi_modeling_rfe.csv")

In [86]:
print(dataset_pca.shape)

print(dataset_MI.shape)

print(dataset_rfe.shape)

(113937, 43)
(113929, 16)
(113929, 13)


In [87]:
dataset_pca.head()

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,...,PC_34,PC_35,PC_36,PC_37,PC_38,PC_39,PC_40,EMI,ELA,PROI
0,0.264848,-2.375961,4.737371,-1.289784,-1.597645,-0.716092,-0.455099,0.966594,-0.777869,1.691510,...,-0.122615,0.592142,-0.107246,-0.147897,-0.429546,-0.684228,-1.654795,1496.764570,63034.0,0.105405
1,-3.329534,-1.963003,-2.070791,1.734427,1.287354,-0.081859,0.257172,0.402567,1.629472,0.623568,...,-0.011336,-0.619403,-0.238048,-0.375699,-0.499301,-0.312770,-0.494449,0.000000,43120.0,0.205405
2,3.869908,-2.294166,1.496543,0.588909,-0.053028,-0.984381,0.746791,3.616560,-1.008640,-0.169295,...,-0.637675,1.292464,-0.503933,-0.084999,0.317305,0.016442,-1.282699,825.406311,32711.0,0.105405
3,-4.133009,-2.363473,-0.671914,-1.507005,0.258205,-1.518471,-1.467166,-3.851630,-0.281209,-0.150314,...,0.171525,-0.460800,0.283255,0.052376,-0.087459,0.117527,0.074013,413.021192,45064.0,0.105405
4,-0.684132,3.371442,-1.088429,0.480719,0.822029,-0.236891,0.607664,0.918144,1.823338,0.396344,...,0.340792,0.039582,-1.054751,-0.016722,0.502393,0.085913,0.124266,326.288414,127590.0,0.205405


In [88]:
dataset_MI.head()

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11,PC_12,PC_13,Target_1,Target_2,Target_3
0,0.174164,-0.145373,1.505678,-0.123672,-0.090143,1.553066,-0.600185,-0.668165,-0.419864,-0.464921,0.300707,0.034815,0.157519,1496.764570,63034.0,0.105405
1,0.266229,-1.318093,-0.873168,-1.530137,-1.474288,-0.763162,-1.351854,-1.228288,-1.419949,-1.347235,0.241026,-0.520337,0.158339,0.000000,43120.0,0.205405
2,-0.854399,-0.145373,0.000755,-0.123672,-0.090143,-0.025656,0.768925,0.794752,1.098553,1.099182,-0.774107,-0.612733,-0.836270,825.406311,32711.0,0.105405
3,0.266229,-1.318093,0.200431,-1.328670,-1.384715,0.242238,-1.279371,-1.164558,-1.333606,-1.275045,0.254104,-0.467665,0.165911,413.021192,45064.0,0.105405
4,1.066791,0.350873,-0.284549,-0.163205,0.275920,-0.378995,0.211885,0.339807,0.271585,0.210183,1.512680,1.632873,1.168902,326.288414,127590.0,0.205405


In [89]:
dataset_MI.rename(columns={dataset_MI.columns[-3]: 'EMI', dataset_MI.columns[-2]: 'ELA', dataset_MI.columns[-1]: 'PROI'}, inplace=True)

In [90]:
dataset_MI.head()

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11,PC_12,PC_13,EMI,ELA,PROI
0,0.174164,-0.145373,1.505678,-0.123672,-0.090143,1.553066,-0.600185,-0.668165,-0.419864,-0.464921,0.300707,0.034815,0.157519,1496.764570,63034.0,0.105405
1,0.266229,-1.318093,-0.873168,-1.530137,-1.474288,-0.763162,-1.351854,-1.228288,-1.419949,-1.347235,0.241026,-0.520337,0.158339,0.000000,43120.0,0.205405
2,-0.854399,-0.145373,0.000755,-0.123672,-0.090143,-0.025656,0.768925,0.794752,1.098553,1.099182,-0.774107,-0.612733,-0.836270,825.406311,32711.0,0.105405
3,0.266229,-1.318093,0.200431,-1.328670,-1.384715,0.242238,-1.279371,-1.164558,-1.333606,-1.275045,0.254104,-0.467665,0.165911,413.021192,45064.0,0.105405
4,1.066791,0.350873,-0.284549,-0.163205,0.275920,-0.378995,0.211885,0.339807,0.271585,0.210183,1.512680,1.632873,1.168902,326.288414,127590.0,0.205405


In [91]:
dataset_rfe.head()

,Feature_62,Feature_1,Feature_43,Feature_63,Feature_48,Feature_41,Feature_50,Feature_40,Feature_49,Feature_3,EMI,ELA,PROI
0,0.034815,-0.462857,-0.337589,0.157519,0.300707,-0.377483,1.553066,-0.191677,1.505678,-0.668165,1496.764570,63034.0,0.105405
1,-0.520337,-0.462857,0.069122,0.158339,0.241026,0.369516,-0.763162,-0.172840,-0.873168,-1.228288,0.000000,43120.0,0.205405
2,-0.612733,-0.462857,-0.471302,-0.836270,-0.774107,1.863515,-0.025656,-0.398888,0.000755,0.794752,825.406311,32711.0,0.105405
3,-0.467665,-0.462857,-0.365446,0.165911,0.254104,-0.377483,0.242238,-0.229352,0.200431,-1.164558,413.021192,45064.0,0.105405
4,1.632873,-0.462857,0.531547,1.168902,1.512680,-1.124482,-0.378995,-0.022141,-0.284549,0.339807,326.288414,127590.0,0.205405


In [92]:
dataset_pca.isna().sum().sum()

344

In [93]:
dataset_pca = dataset_pca.dropna()

In [94]:
dataset_pca.isna().sum().sum()

0

In [95]:
dataset_MI.isna().sum().sum()

0

In [96]:
dataset_rfe.isna().sum().sum()

0

**1. Applying basic ML models for dataset with features selected using PCA.**

In [97]:
# Splitting the data into independent (X) and dependent variables (y).

X_pca = dataset_pca.drop(columns=['EMI', 'ELA', 'PROI'], axis = 1)
y_pca = dataset_pca[['EMI', 'ELA', 'PROI']]

In [98]:
X_pca.head()

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,...,PC_31,PC_32,PC_33,PC_34,PC_35,PC_36,PC_37,PC_38,PC_39,PC_40
0,0.264848,-2.375961,4.737371,-1.289784,-1.597645,-0.716092,-0.455099,0.966594,-0.777869,1.691510,...,1.344877,-0.028122,0.423377,-0.122615,0.592142,-0.107246,-0.147897,-0.429546,-0.684228,-1.654795
1,-3.329534,-1.963003,-2.070791,1.734427,1.287354,-0.081859,0.257172,0.402567,1.629472,0.623568,...,0.432239,-0.038646,-0.096128,-0.011336,-0.619403,-0.238048,-0.375699,-0.499301,-0.312770,-0.494449
2,3.869908,-2.294166,1.496543,0.588909,-0.053028,-0.984381,0.746791,3.616560,-1.008640,-0.169295,...,0.842702,-0.813310,0.193225,-0.637675,1.292464,-0.503933,-0.084999,0.317305,0.016442,-1.282699
3,-4.133009,-2.363473,-0.671914,-1.507005,0.258205,-1.518471,-1.467166,-3.851630,-0.281209,-0.150314,...,0.606711,0.203126,-1.226017,0.171525,-0.460800,0.283255,0.052376,-0.087459,0.117527,0.074013
4,-0.684132,3.371442,-1.088429,0.480719,0.822029,-0.236891,0.607664,0.918144,1.823338,0.396344,...,0.325631,0.230086,-0.200241,0.340792,0.039582,-1.054751,-0.016722,0.502393,0.085913,0.124266


In [99]:
y_pca.head()

,EMI,ELA,PROI
0,1496.764570,63034.0,0.105405
1,0.000000,43120.0,0.205405
2,825.406311,32711.0,0.105405
3,413.021192,45064.0,0.105405
4,326.288414,127590.0,0.205405


In [100]:
dataset_pca.head(20)

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,...,PC_34,PC_35,PC_36,PC_37,PC_38,PC_39,PC_40,EMI,ELA,PROI
0,0.264848,-2.375961,4.737371,-1.289784,-1.597645,-0.716092,-0.455099,0.966594,-0.777869,1.691510,...,-0.122615,0.592142,-0.107246,-0.147897,-0.429546,-0.684228,-1.654795,1496.764570,63034.0,0.105405
1,-3.329534,-1.963003,-2.070791,1.734427,1.287354,-0.081859,0.257172,0.402567,1.629472,0.623568,...,-0.011336,-0.619403,-0.238048,-0.375699,-0.499301,-0.312770,-0.494449,0.000000,43120.0,0.205405
2,3.869908,-2.294166,1.496543,0.588909,-0.053028,-0.984381,0.746791,3.616560,-1.008640,-0.169295,...,-0.637675,1.292464,-0.503933,-0.084999,0.317305,0.016442,-1.282699,825.406311,32711.0,0.105405
3,-4.133009,-2.363473,-0.671914,-1.507005,0.258205,-1.518471,-1.467166,-3.851630,-0.281209,-0.150314,...,0.171525,-0.460800,0.283255,0.052376,-0.087459,0.117527,0.074013,413.021192,45064.0,0.105405
4,-0.684132,3.371442,-1.088429,0.480719,0.822029,-0.236891,0.607664,0.918144,1.823338,0.396344,...,0.340792,0.039582,-1.054751,-0.016722,0.502393,0.085913,0.124266,326.288414,127590.0,0.205405
5,-5.314834,2.372446,-2.820566,2.678798,0.360996,0.098589,0.035162,-1.227376,-0.854605,0.018945,...,-0.391940,1.511522,-0.335304,-0.237395,0.941788,0.597594,0.036185,46.266419,133260.0,0.255405
6,3.773871,0.239875,-2.028054,-1.214925,-0.686469,0.585285,0.235952,0.234394,-0.158794,-0.567046,...,-0.298502,-0.523605,-0.139646,0.616491,-0.954032,0.740011,0.453348,163.901632,32290.0,0.205405
7,0.776262,-0.638985,-0.095075,-2.094947,0.839008,-0.047619,0.090738,0.151658,1.462528,0.902347,...,-0.511865,-1.115256,-0.606508,-0.106265,-0.896890,-0.007573,-0.290691,395.421178,82684.0,0.255405
8,-6.318251,-1.838507,-2.759189,1.902683,-0.139716,0.702797,-0.784828,-2.069267,1.419786,0.223854,...,-0.507640,-0.073472,-0.538123,-0.212032,1.068972,0.986907,0.556737,35.781421,32644.0,0.205405
9,-6.408849,-1.909146,-2.732292,1.900168,-0.109903,0.663138,-0.802591,-2.073740,1.399565,0.159160,...,-0.570424,0.047046,-0.547887,-0.334893,1.160860,0.785518,0.722542,35.781421,32644.0,0.205405


In [101]:
# Split the data into training and testing sets
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y_pca, test_size=0.2, random_state=42)

# Initialize models
linear_reg_pca = LinearRegression()
ridge_reg_pca = Ridge(alpha=1.0, random_state=42)  # alpha controls the L2 penalty strength
lasso_reg_pca = Lasso(alpha=0.01, random_state=42)  # alpha controls the L1 penalty strength

# Fit models

linear_reg_pca.fit(X_train_pca, y_train_pca)
ridge_reg_pca.fit(X_train_pca, y_train_pca)
lasso_reg_pca.fit(X_train_pca, y_train_pca)

# Predict on the test set
y_pred_lr_pca = linear_reg_pca.predict(X_test_pca)
y_pred_ridge_pca = ridge_reg_pca.predict(X_test_pca)
y_pred_lasso_pca = lasso_reg_pca.predict(X_test_pca)

# Evaluate models using Mean Squared Error (MSE)
mse_lr_pca = mean_squared_error(y_test_pca, y_pred_lr_pca, multioutput='raw_values')
mse_ridge_pca = mean_squared_error(y_test_pca, y_pred_ridge_pca, multioutput='raw_values')
mse_lasso_pca = mean_squared_error(y_test_pca, y_pred_lasso_pca, multioutput='raw_values')

# Evaluate models using R squared value (R2)
r2_lr_pca = r2_score(y_test_pca, y_pred_lr_pca, multioutput='raw_values')
r2_ridge_pca = r2_score(y_test_pca, y_pred_ridge_pca, multioutput='raw_values')
r2_lasso_pca = r2_score(y_test_pca, y_pred_lasso_pca, multioutput='raw_values')

# Printing the evaluation metrics (MSE and R2)

print("\nMean Squared Error for Linear Regression:", mse_lr_pca)

print("Mean Squared Error for Ridge Regression:", mse_ridge_pca)

print("Mean Squared Error for Lasso Regression:", mse_lasso_pca)

print("\n")

print("\nR2 for Linear Regression:", r2_lr_pca)

print("R2 for Ridge Regression:", r2_ridge_pca)

print("R2 for Lasso Regression:", r2_lasso_pca)


Mean Squared Error for Linear Regression: [4.93034782e+05 3.36638184e+09 5.32883437e-03]
Mean Squared Error for Ridge Regression: [4.93034765e+05 3.36638175e+09 5.32883427e-03]
Mean Squared Error for Lasso Regression: [4.93033172e+05 3.36638184e+09 5.47208959e-03]



R2 for Linear Regression: [0.06070848 0.06813231 0.04918444]
R2 for Ridge Regression: [0.06070852 0.06813233 0.04918445]
R2 for Lasso Regression: [0.06071155 0.06813231 0.02362363]


**2. Applying basic ML models for dataset with features selected using Mutual Information.**

In [102]:
dataset_MI.head()

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11,PC_12,PC_13,EMI,ELA,PROI
0,0.174164,-0.145373,1.505678,-0.123672,-0.090143,1.553066,-0.600185,-0.668165,-0.419864,-0.464921,0.300707,0.034815,0.157519,1496.764570,63034.0,0.105405
1,0.266229,-1.318093,-0.873168,-1.530137,-1.474288,-0.763162,-1.351854,-1.228288,-1.419949,-1.347235,0.241026,-0.520337,0.158339,0.000000,43120.0,0.205405
2,-0.854399,-0.145373,0.000755,-0.123672,-0.090143,-0.025656,0.768925,0.794752,1.098553,1.099182,-0.774107,-0.612733,-0.836270,825.406311,32711.0,0.105405
3,0.266229,-1.318093,0.200431,-1.328670,-1.384715,0.242238,-1.279371,-1.164558,-1.333606,-1.275045,0.254104,-0.467665,0.165911,413.021192,45064.0,0.105405
4,1.066791,0.350873,-0.284549,-0.163205,0.275920,-0.378995,0.211885,0.339807,0.271585,0.210183,1.512680,1.632873,1.168902,326.288414,127590.0,0.205405


In [103]:
# Splitting the data into independent (X) and dependent variables (y).

X_MI = dataset_MI.drop(columns=['EMI', 'ELA', 'PROI'], axis = 1)
y_MI = dataset_MI[['EMI', 'ELA', 'PROI']]

In [104]:
X_MI.head()

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11,PC_12,PC_13
0,0.174164,-0.145373,1.505678,-0.123672,-0.090143,1.553066,-0.600185,-0.668165,-0.419864,-0.464921,0.300707,0.034815,0.157519
1,0.266229,-1.318093,-0.873168,-1.530137,-1.474288,-0.763162,-1.351854,-1.228288,-1.419949,-1.347235,0.241026,-0.520337,0.158339
2,-0.854399,-0.145373,0.000755,-0.123672,-0.090143,-0.025656,0.768925,0.794752,1.098553,1.099182,-0.774107,-0.612733,-0.836270
3,0.266229,-1.318093,0.200431,-1.328670,-1.384715,0.242238,-1.279371,-1.164558,-1.333606,-1.275045,0.254104,-0.467665,0.165911
4,1.066791,0.350873,-0.284549,-0.163205,0.275920,-0.378995,0.211885,0.339807,0.271585,0.210183,1.512680,1.632873,1.168902


In [105]:
y_MI.head()

,EMI,ELA,PROI
0,1496.764570,63034.0,0.105405
1,0.000000,43120.0,0.205405
2,825.406311,32711.0,0.105405
3,413.021192,45064.0,0.105405
4,326.288414,127590.0,0.205405


In [106]:
# Split the data into training and testing sets
X_train_MI, X_test_MI, y_train_MI, y_test_MI = train_test_split(X_MI, y_MI, test_size=0.2, random_state=42)

# Initialize models
linear_reg_MI = LinearRegression()
ridge_reg_MI = Ridge(alpha=1.0, random_state=42)  # alpha controls the L2 penalty strength
lasso_reg_MI = Lasso(alpha=0.01, random_state=42)  # alpha controls the L1 penalty strength

# Fit models

linear_reg_MI.fit(X_train_MI, y_train_MI)
ridge_reg_MI.fit(X_train_MI, y_train_MI)
lasso_reg_MI.fit(X_train_MI, y_train_MI)

# Predict on the test set
y_pred_lr_MI = linear_reg_MI.predict(X_test_MI)
y_pred_ridge_MI = linear_reg_MI.predict(X_test_MI)
y_pred_lasso_MI = linear_reg_MI.predict(X_test_MI)

# Evaluate models using Mean Squared Error (MSE)
mse_lr_MI = mean_squared_error(y_test_MI, y_pred_lr_MI, multioutput='raw_values')
mse_ridge_MI = mean_squared_error(y_test_MI, y_pred_ridge_MI, multioutput='raw_values')
mse_lasso_MI = mean_squared_error(y_test_MI, y_pred_lasso_MI, multioutput='raw_values')

# Evaluate models using R squared value (R2)
r2_lr_MI = r2_score(y_test_MI, y_pred_lr_MI, multioutput='raw_values')
r2_ridge_MI = r2_score(y_test_MI, y_pred_ridge_MI, multioutput='raw_values')
r2_lasso_MI = r2_score(y_test_MI, y_pred_lasso_MI, multioutput='raw_values')

# Printing the evaluation metrics (MSE and R2)

print("\nMean Squared Error for Linear Regression:", mse_lr_MI)

print("Mean Squared Error for Ridge Regression:", mse_ridge_MI)

print("Mean Squared Error for Lasso Regression:", mse_lasso_MI)

print("\n")

print("\nR2 for Linear Regression:", r2_lr_MI)

print("R2 for Ridge Regression:", r2_ridge_MI)

print("R2 for Lasso Regression:", r2_lasso_MI)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.858e+09, tolerance: 4.690e+06
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.195e+13, tolerance: 3.305e+10
  model = cd_fast.enet_coordinate_descent(



Mean Squared Error for Linear Regression: [4.61581405e+04 3.00250593e+08 2.00542505e-03]
Mean Squared Error for Ridge Regression: [4.61581405e+04 3.00250593e+08 2.00542505e-03]
Mean Squared Error for Lasso Regression: [4.61581405e+04 3.00250593e+08 2.00542505e-03]



R2 for Linear Regression: [0.9138048  0.91933516 0.64471437]
R2 for Ridge Regression: [0.9138048  0.91933516 0.64471437]
R2 for Lasso Regression: [0.9138048  0.91933516 0.64471437]


**3. Applying basic ML models for dataset with features selected using Random Forest Regressor.**

In [107]:
# Splitting the data into independent (X) and dependent variables (y).

X_rfe = dataset_rfe.drop(columns=['EMI', 'ELA', 'PROI'], axis = 1)
y_rfe = dataset_rfe[['EMI', 'ELA', 'PROI']]

In [108]:
# Split the data into training and testing sets
X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe = train_test_split(X_rfe, y_rfe, test_size=0.2, random_state=42)

# Initialize models
linear_reg_rfe = LinearRegression()
ridge_reg_rfe = Ridge(alpha=1.0, random_state=42)  # alpha controls the L2 penalty strength
lasso_reg_rfe = Lasso(alpha=0.01, random_state=42)  # alpha controls the L1 penalty strength

# Fit models

linear_reg_rfe.fit(X_train_rfe, y_train_rfe)
ridge_reg_rfe.fit(X_train_rfe, y_train_rfe)
lasso_reg_rfe.fit(X_train_rfe, y_train_rfe)

# Predict on the test set
y_pred_lr_rfe = linear_reg_rfe.predict(X_test_rfe)
y_pred_ridge_rfe = ridge_reg_rfe.predict(X_test_rfe)
y_pred_lasso_rfe = lasso_reg_rfe.predict(X_test_rfe)

# Evaluate models using Mean Squared Error (MSE)
mse_lr_rfe = mean_squared_error(y_test_rfe, y_pred_lr_rfe, multioutput='raw_values')
mse_ridge_rfe = mean_squared_error(y_test_rfe, y_pred_ridge_rfe, multioutput='raw_values')
mse_lasso_rfe = mean_squared_error(y_test_rfe, y_pred_lasso_rfe, multioutput='raw_values')

# Evaluate models using R squared value (R2)
r2_lr_rfe = r2_score(y_test_rfe, y_pred_lr_rfe, multioutput='raw_values')
r2_ridge_rfe = r2_score(y_test_rfe, y_pred_ridge_rfe, multioutput='raw_values')
r2_lasso_rfe = r2_score(y_test_rfe, y_pred_lasso_rfe, multioutput='raw_values')

# Printing the evaluation metrics (MSE and R2)

print("\nMean Squared Error for Linear Regression:", mse_lr_rfe)

print("Mean Squared Error for Ridge Regression:", mse_ridge_rfe)

print("Mean Squared Error for Lasso Regression:", mse_lasso_rfe)

print("\n")

print("\nR2 for Linear Regression:", r2_lr_rfe)

print("R2 for Ridge Regression:", r2_ridge_rfe)

print("R2 for Lasso Regression:", r2_lasso_rfe)


Mean Squared Error for Linear Regression: [4.85101991e+04 2.22004640e+08 2.21929344e-03]
Mean Squared Error for Ridge Regression: [4.85103741e+04 2.22002629e+08 2.21929725e-03]
Mean Squared Error for Lasso Regression: [4.85105981e+04 2.22004601e+08 2.65894480e-03]



R2 for Linear Regression: [0.90941259 0.94035659 0.60682496]
R2 for Ridge Regression: [0.90941227 0.94035713 0.60682428]
R2 for Lasso Regression: [0.90941185 0.9403566  0.52893533]


**Observations**

**1. PCA Features Dataset**

      a. Linear Regression, Ridge and Lasso, all have High MSE values for all 3 target variables.
      b. Very low R² values, suggesting PCA features are not effective in explaining target variability.

**2. Mutual Information Features Dataset**

      a. Linear Regression, Ridge and Lasso has identical performance metrics so they perform equally well.
      b. All the 3 models consistently has low MSE and high R² for all targets.
      c. Mutual Information (MI) features perform significantly better than PCA.

**3. Random Forest Regressor Features Dataset**

      a. The performance of Linear Regression, Ridge and Lasso based on MSE and R² is comparable, though Lasso slightly underperforms for the third target (PROI).
      b. Random Forest features also perform well, but MI features slightly outperform them based on R² for the third target (PROI).

**Conclusion**

**1. Feature Selection Method:**

    a. Mutual Information Features are the best choice, as they achieve consistently low MSE and high R² across all targets.
    b. Random Forest Features are close in performance and can be considered a secondary choice.

**2. Regressor Model:**

    a. Linear Regression is the simplest model and performs equally well as Ridge and Lasso when Mutual Information features are used.
    b. Ridge and Lasso are useful for additional regularization but provide no clear advantage in this case.


**After evaluating different feature selection methods and regression models, the Mutual Information features combined with Linear Regression emerge as the optimal choice, demonstrating superior performance across all target variables compared to the other approaches.**

In [109]:
# Define a pipeline for Polynomial Features + Linear Regression
pipeline = Pipeline([
    ('poly_features', PolynomialFeatures()),
    ('linear_reg', LinearRegression())
])

# Define the hyperparameters to tune (degree of polynomial features)
param_grid = {
    'poly_features__degree': [1, 2, 3, 4],  # Test different polynomial degrees
    'poly_features__include_bias': [False, True]  # Include/exclude bias term
}

# Set up GridSearchCV to tune the pipeline
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # Use MSE as the evaluation metric
    n_jobs=-1
)

# Perform Grid Search on the training data
grid_search.fit(X_train_MI, y_train_MI)

# Get the best model from Grid Search
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred_best = best_model.predict(X_test_MI)

# Calculate performance metrics for the best model
mse_best = mean_squared_error(y_test_MI, y_pred_best, multioutput='raw_values')
r2_best = r2_score(y_test_MI, y_pred_best, multioutput='raw_values')

# Print the results
print("\nBest Hyperparameters:", grid_search.best_params_)
print("\nMean Squared Error for Best Linear Regression Model:", mse_best)
print("R2 Score for Best Linear Regression Model:", r2_best)


Best Hyperparameters: {'poly_features__degree': 2, 'poly_features__include_bias': True}

Mean Squared Error for Best Linear Regression Model: [7.92348115e+01 1.36952877e+08 1.18036583e-03]
R2 Score for Best Linear Regression Model: [0.99985204 0.96320646 0.79088372]


In [110]:
# Saving the model

joblib.dump(best_model, 'finalized_linear_regression_model.pkl')

# Print confirmation message
print("Best Linear Regression model has been saved successfully!")

Best Linear Regression model has been saved successfully!
